In [ ]:
import os
import json
import math
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') 
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import torchvision
from torchvision.datasets import FashionMNIST
from torchvision import transforms

device = torch.device("cuda")


# data preprocessing

In [ ]:
transform = transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
     )


batch_size = 32
dataset = torchvision.datasets.ImageFolder("/dataset_folder/", transform=transform)

train_dataset,test_set = torch.utils.data.random_split(dataset, [0.8,0.2]) 
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=2)
test_loader = data.DataLoader(test_set, batch_size=128, shuffle=False, drop_last=False, num_workers=2)

# GAN model

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.linear1 = nn.Linear(latent_dim, 512)  
        self.main = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):       
        output = self.linear1(input)
        output = output.view(output.size(0), 512, 1, 1)  
        output = self.main(output)
        return output


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

# parameters

In [ ]:
# Models
latent_dim = 32
generator = Generator(latent_dim)
discriminator = Discriminator()

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    generator.to(device)
    discriminator.to(device)

# Optimizers
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
generator_scheduler = optim.lr_scheduler.ExponentialLR(optimizer=generator_optimizer, gamma=0.99)

discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)
discriminator_scheduler = optim.lr_scheduler.ExponentialLR(optimizer=discriminator_optimizer, gamma=0.99)

# loss
criterion = nn.BCELoss()

In [ ]:
fixed_noise = torch.randn(16, latent_dim,1,1).to(device)

# training

In [ ]:
G_losses = []
D_losses = []
num_epochs = 100
for epoch in range(num_epochs):
    discriminator_fake_acc = []
    discriminator_real_acc = []
    for i, data in enumerate(train_loader, 0):
        discriminator_optimizer.zero_grad()
        real_images = data[0].to(device)
        b_size = real_images.size(0)
        label = torch.ones((b_size,), dtype=torch.float).to(device) # Setting labels for real images
        output = discriminator(real_images).view(-1)
        error_discriminator_real = criterion(output, label)
        discriminator_real_acc.append(output.mean().item())
        noise = torch.randn(b_size, latent_dim,1,1).to(device)
        fake_images = generator(noise).to(device)
        label_fake = torch.zeros((b_size,), dtype=torch.float).to(device)
        output = discriminator(fake_images.detach()).view(-1)
        error_discriminator_fake = criterion(output, label_fake)
        discriminator_fake_acc.append(output.mean().item())
        error_discriminator = error_discriminator_real + error_discriminator_fake
        error_discriminator.backward()
        discriminator_optimizer.step()

        generator_optimizer.zero_grad()
        label = torch.ones((b_size,), dtype=torch.float).to(device)  # fake labels are real for generator cost
        output = discriminator(fake_images).view(-1)
        error_generator = criterion(output, label)
        error_generator.backward()
        D_G_z2 = output.mean().item()
        generator_optimizer.step()

        G_losses.append(error_generator.item())
        D_losses.append(error_discriminator.item())

    print(f"Epoch: {epoch}, discrimiantor fake error: {np.mean(discriminator_fake_acc):.3}, discriminator real acc: {np.mean(discriminator_real_acc):.3}")
    generator_scheduler.step()
    discriminator_scheduler.step()
    if epoch % 10 == 0:
        with torch.no_grad():
            fake = generator(fixed_noise).detach().cpu()
        grid = torchvision.utils.make_grid(fake, normalize=True)
        grid = grid.permute(1, 2, 0)
        plt.figure(figsize=(10,10))
        plt.title(f"Generations")
        plt.imshow(grid)
        plt.axis('off')
        plt.show()

# generate new data

In [ ]:
from torchvision.utils import save_image
from PIL import Image

random_noise = torch.randn(1000, latent_dim).to(device)
with torch.no_grad():
        generated_imgs = generator(random_noise).detach().cpu()

torch.save(generated_imgs,"poniedzialek_haczynski_zemla.pt")
!mkdir  -p /content/ssne_4/output/images/
!mkdir  -p /content/ssne_4/output/real_images/
for i in range(len(generated_imgs)):
    img1 = generated_imgs[i]
    grid = torchvision.utils.make_grid(img1, normalize=True)
    grid = grid.permute(1, 2, 0)
    grid = grid.cpu().numpy() * 255
    grid = grid.astype(np.uint8)
    grid = Image.fromarray(grid)
    grid.save("/images/"+str(i)+".jpeg")

from torchvision.datasets import ImageFolder
from PIL import Image
import os
from torchvision.transforms import ToPILImage